In [28]:
# import libraries

import datetime
import pickle
import nltk
import re
import scipy
import pandas as pd
from scipy import sparse
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from utils import get_weather
from utils import text_processing

### Get and preprocess ISW files

In [29]:
# get article from yesterday
today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)

yesterday_day = yesterday.day
yesterday_month = yesterday.month
yesterday_year = yesterday.year

In [30]:
file = text_processing.get_article_from_yesterday(yesterday_day,yesterday_month,yesterday_year)

In [31]:
data = text_processing.read_html(file)

In [32]:
def preprocess_all_text(data):
    pattern = "\[(\d+)\]"
    data['main_html_v1'] = data['main_html'].apply(lambda x: re.sub(pattern,"",str(x)))
    data['main_html_v2'] = data['main_html_v1'].apply(lambda x: re.sub(r'http(\S+.*\s)',"",x))
    data['main_html_v3'] = data['main_html_v2'].apply(lambda x: re.sub(r'2022|2023|©2022|©2023|\xa0|\n',"",x))
    data['main_html_v4'] = data['main_html_v3'].apply(lambda x: BeautifulSoup(x).text)
    data['main_html_v5'] = data['main_html_v4'].apply(lambda x: text_processing.remove_names_and_dates(x))
    
    return data

In [33]:
data_preprocessed = preprocess_all_text(data)

In [34]:
data_preprocessed = data_preprocessed.drop(['main_html_v1','main_html_v2','main_html_v3','main_html_v4'],axis=1)

In [35]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dimai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [36]:
data_preprocessed['report_text_lemm'] = data_preprocessed['main_html_v5'].apply(lambda x: text_processing.preprocess(x,"lemm"))
data_preprocessed['report_text_stemm'] = data_preprocessed['main_html_v5'].apply(lambda x: text_processing.preprocess(x,"stemm"))

In [37]:
data_preprocessed.head()

,date,short_url,title,text_title,full_url,main_html,main_html_v5,report_text_lemm,report_text_stemm
0,2023-04-26,russian_offensive_campaign_assessment_April_26...,404 | Institute for the Study of War,404,/404,[[[<h2>We're Sorry. The page you're looking fo...,We're Sorry. The page you're looking for wasn'...,sorri page look found search formsearch,sorri page look found search formsearch


In [38]:
docs = data_preprocessed['report_text_lemm'].tolist()

### ISW vectorize

In [39]:
cv = CountVectorizer()
word_count_vector = cv.fit_transform(docs)

word_count_vector.shape

tfidf_transformer = TfidfTransformer(smooth_idf = True, use_idf = True)
tfidf_transformer.fit(word_count_vector)

tf_idf_vector = tfidf_transformer.transform(word_count_vector)

In [40]:
tf_idf_vector

<1x6 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [41]:
feature_names = cv.get_feature_names_out()
tf_idf_vector

<1x6 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [42]:
data_preprocessed['keywords'] = data_preprocessed['report_text_stemm'].apply(lambda x: text_processing.convert_doc_to_vector(x,feature_names,tf_idf_vector))

In [43]:
data_preprocessed['keywords'][0]

{'sorri': 0.408,
 'search': 0.408,
 'page': 0.408,
 'look': 0.408,
 'found': 0.408,
 'formsearch': 0.408}

In [44]:
data_preprocessed.head()

,date,short_url,title,text_title,full_url,main_html,main_html_v5,report_text_lemm,report_text_stemm,keywords
0,2023-04-26,russian_offensive_campaign_assessment_April_26...,404 | Institute for the Study of War,404,/404,[[[<h2>We're Sorry. The page you're looking fo...,We're Sorry. The page you're looking for wasn'...,sorri page look found search formsearch,sorri page look found search formsearch,"{'sorri': 0.408, 'search': 0.408, 'page': 0.40..."


#### Part of script: Final preprocessing

In [45]:
data_preprocessed["date_datetime"] = pd.to_datetime(data_preprocessed["date"])
data_preprocessed['date_tomorrow_datetime'] = data_preprocessed['date_datetime'].apply(lambda x: x+datetime.timedelta(days=1))
data_preprocessed = data_preprocessed.rename(columns = {"date_datetime":"report_date"})

In [46]:
data_preprocessed.head()

,date,short_url,title,text_title,full_url,main_html,main_html_v5,report_text_lemm,report_text_stemm,keywords,report_date,date_tomorrow_datetime
0,2023-04-26,russian_offensive_campaign_assessment_April_26...,404 | Institute for the Study of War,404,/404,[[[<h2>We're Sorry. The page you're looking fo...,We're Sorry. The page you're looking for wasn'...,sorri page look found search formsearch,sorri page look found search formsearch,"{'sorri': 0.408, 'search': 0.408, 'page': 0.40...",2023-04-26,2023-04-27


In [47]:
data_vectorised = tf_idf_vector.toarray()
vectors_df = pd.DataFrame(data_vectorised)
vectors_df['date'] = pd.to_datetime(today)


In [48]:
vectors_df.head()

,0,1,2,3,4,5,date
0,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248,2023-04-27


In [49]:
df_isw_short = data_preprocessed[['date','report_text_lemm','keywords','date_tomorrow_datetime']]

In [50]:
df_isw_short.head()

,date,report_text_lemm,keywords,date_tomorrow_datetime
0,2023-04-26,sorri page look found search formsearch,"{'sorri': 0.408, 'search': 0.408, 'page': 0.40...",2023-04-27


### Predict

In [ ]:
tfidf = pickle.load(open("models/tfidf_transformer_v1.pkl","rb"))
cv = pickle.load(open("models/count_vectorizer_v1.pkl","rb"))
model = pickle.load(open("models/training_models/4_rf_3.1f.pkl","rb"))

tfidf_vector = scipy.sparse.load_npz('data/matrix/tfidf_vector_train.npz')

In [ ]:
cities = ['Крим','Вінниччина','Волинь','Дніпропетровщина','Донеччина','Житомирщина','Закарпаття', 'Запоріжжя',
          'Івано-Франківщина','Київщина','Кіровоградщина','Луганщина','Львівщина','Миколаївщина','Одещина','Полтавщина',
          'Рівненщина','Сумщина','Тернопільщина','Харківщина','Херсонщина','Хмельниччина','Черкащина',
          'Буковина','Чернігівщина']

date = today = datetime.date.today()
result = {}
for city in cities:

    df_weather_final = get_weather.get_weather_for_12_hours(city,date)

    # merge
    df_weather_final['key']=1
    df_isw_short['key']=1
    df_all = df_weather_final.merge(df_isw_short, how = 'left', left_on = 'key', right_on = 'key')

    # drop
    to_drop=['key','date','date_tomorrow_datetime','keywords','report_text_lemm']
    if 'sunrise' in df_all.columns:
        exceptions = ['sunset','sunrise']
        to_drop.extend(exceptions)
    df_weather_matrix_v1 = df_all.drop(to_drop, axis = 1)

    # final dataset
    df_weather_matrix_v1= df_weather_matrix_v1[['day_tempmax', 'day_tempmin', 'day_temp', 'day_dew', 'day_humidity',
           'day_precip', 'day_precipcover', 'day_solarradiation',
           'day_solarenergy', 'day_uvindex', 'hour_temp', 'hour_humidity',
           'hour_dew', 'hour_precip', 'hour_precipprob', 'hour_snow',
           'hour_snowdepth', 'hour_windgust', 'hour_windspeed', 'hour_winddir',
           'hour_pressure', 'hour_visibility', 'hour_cloudcover',
           'hour_solarradiation', 'hour_uvindex', 'hour_severerisk','region_id','hour_datetimeEpoch']]

    cv_vector_model = cv.transform(df_all['report_text_lemm'].values.astype('U'))
    #TF_IDF_MODEL = tfidf.transform(cv_vector_model)

    df_weather_matrix_v1_csr = scipy.sparse.csr_matrix(df_weather_matrix_v1)
    df_all_data_csr = scipy.sparse.hstack((df_weather_matrix_v1_csr, tfidf_vector), format='csr')

    #predict
    predicted = model.predict(df_all_data_csr)
    current_time = pd.Timestamp.now()

    hours = []
    for i in range(12):
        hour = current_time + datetime.timedelta(hours=i)
        hours.append(hour) 

    result[city] = dict(zip(hours, predicted))
    

In [ ]:
result

In [ ]:
result = pd.DataFrame(result)
result.to_csv('results.txt', sep='\t', index=False)